# 2023-08-Hilary

In [5]:
import datetime
from datetime import timezone
from geopandas import GeoDataFrame
from pystac_client import Client

earth_search = Client.open("https://earth-search.aws.element84.com/v1/")
formed = datetime.datetime(2023, 8, 16, tzinfo=timezone.utc)
dissipated = datetime.datetime(2023, 8, 21, tzinfo=timezone.utc)

# https://www.nhc.noaa.gov/gis/archive_besttrack_results.php?id=ep09&year=2023&name=Hurricane%20HILARY

In [ ]:
search = earth_search.search(
    collections=["landsat-c2-l2"],
)